## Import Required Libraries
Running cell below will import the required libraries being used in the rest of the cells of this notebook

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def get_params(exp_no='E1'):
    model_type              = 'real'
    model_name   = 'yolo11s'     
    background_type         = 'gray'
    file_name = 'exp_E4'

    return model_type, model_name,background_type,file_name

def get_exp_header(model_type,pretrained_model_type,background_type,file_name):
    print('='*95)
    print('| ',f'{"MODEL_TYPE":15}','|',f'{"PRETRAINED_MODEL":22}','|',f'{"BACKGROUND_TYPE":22}','|',f'{"FILENAME":22}','|')
    print('-'*95)
    print('| ',f'{model_type:15}','|',f'{pretrained_model_type:22}','|',f'{background_type:22}','|',f'{file_name:22}','|')

### PARAMS & Paths
Below cell contains the parameters switching between the prediction model, background replacement strategy, and other few parameters

In [ ]:
exp_no = 'E4'

# use_subset = False
use_subset = True

if use_subset:
    subset_count = 200

model_type,model_name,background_type,file_name = get_params(exp_no=exp_no)
get_exp_header(model_type,model_name,background_type,file_name)

In [ ]:
suffix_full      = f'{model_type}_{background_type}'
DS_name_annotation    = 'ImageNetS'
DS_name_img           = 'ImageNet'
# if select_random_test:
#     subset_type = 'im50'
#     # subset_type = 'im300'
    
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))


codes_path    = os.getcwd() 
results_path    = os.path.join(codes_path,'results')

# if model_type=='real':
    # results_path    = os.path.join(results_path,f'{model_name}_{background_type}')

suffix                     = f'{model_name}_{background_type}'
results_main                   = os.path.join(codes_path,'results')
results_path    = os.path.join(results_path,suffix)

plots_path                     = os.path.join(results_path,'plots')
plotsIoU_path                     = os.path.join(results_path,'plots_IoU')



path_utils    = os.path.abspath(os.path.join(path_notebook,'utils'))
path_html        = results_path


# path_csv          = os.path.join(codes_path,     'csv')
path_csv    = '../results_logits_with_auc_clipped/'
path_boxplot          = os.path.join(codes_path,     'boxplot')
os.makedirs(path_boxplot,              exist_ok=True)

In [ ]:
path_csv,path_utils,path_html,plots_path,plotsIoU_path


## LOAD CSV FILE GENERATED BY `notebooks/N2_Run_experiments.ipynb`

In [ ]:
# csv_filename = f'{path_csv}/csv_exp_{DS_name_img}_{suffix}_im50.csv'

# csv_filename = f'{path_csv}/csv_exp_{DS_name_img}_{suffix}_{subset_type}.csv'
csv_filename = f'{path_csv}/csv_exp_{suffix}_7.csv'

csv_filename,os.path.exists(csv_filename)

In [ ]:
df = pd.read_csv(csv_filename)
df.head(3)

In [ ]:
# #     name,                 color,                  functor
# methods = [ 
#     'BPT-100',
#     'BPT-500',   
#     'BPT-1000',  
#     ]
# methods_pe = [ # if single background
#     'Partition-100', 
#     'Partition-500',  
#     'Partition-1000',  
#     ]
# # methods_aa = [ # if multiple backgrounds
# #     'AA-100',   
# #     'AA-500',
# #     'AA-1000', 
# # ]
# # methods_aa_huge = [
# #     'AA-5000', 
# #     'AA-10000',
# # ]
# methods_lime = [
#     # 'LIME-50',
#     # 'LIME-100',
#     # 'LIME-200',  
#     'LIME-250',  
#     ]
# methods_cam = [
#     'GradCAM',  
#     'aIDG',       
#     'aGradExpl', 
#     ]
# methods += methods_pe if background_type=='gray' and model_type!='ideal' else methods_aa
# if model_type == 'ideal':
#     methods += methods_aa_huge

# methods += methods_lime
# if model_type != 'ideal':
#     methods += methods_cam

In [ ]:
methods = df.method.unique()
methods

In [ ]:
df,model,background = pd.read_csv(csv_filename), model_type,background_type

In [ ]:
files = np.unique(df.image)
print(len(files))


In [ ]:
computed_methods = np.unique(df.method)
num_methods = len(computed_methods)
num_images  = len(np.unique(df.image))

In [ ]:
print(num_methods,computed_methods)

In [ ]:
print(f'Total Images: {num_images}')

In [ ]:
print('model_type:\t', model_type)
print('model_name:\t', model_name)


In [ ]:
df.rename(columns={'auc_IoU': 'au_IoU'}, inplace=True)

In [ ]:
name_remap = {
    'Partition-100': 'PE-100',
    'Partition-500': 'PE-500',
    'Partition-1000': 'PE-1000',
    'BPT-100': '\\bf BPT-100',
    'BPT-500': '\\bf BPT-500',
    'BPT-1000': '\\bf BPT-1000',
    'BPTR-100': '\\bf BPTR-100',
    'BPTR-500': '\\bf BPTR-500',
    'BPTR-1000': '\\bf BPTR-1000',
    'aIDG': '$|$IDG$|$',
    'aGradExpl': '$|$GradExpl$|$',
}
method_remap = {
    'max_IoU': '$max$-$IoU$',
    'au_IoU': '$AU$-$IoU$',
    'aucI_pred': '$\\mathit{AUC}^+$',
    'aucD_pred': '$\\mathit{AUC}^-$',
    'aucI_mse': '$\\mathit{MSE}^+$',
    'aucD_mse': '$\\mathit{MSE}^-$',
    'time_exp': 'log(time)'
}
method_colors = {
    'BPT-100':         '#f2a7c0',
    'BPT-500':         '#db587f',
    'BPT-1000':        '#9d2f4d',
    'BPTR-100':         '#f2a7c0',
    'BPTR-500':         '#db587f',
    'BPTR-1000':        '#9d2f4d',
    'AA-100':   '#4ed1f9',
    'AA-500':   '#00b1f2',
    'AA-1000':  '#008fe8',
    'AA-5000':  '#006bd7',
    'AA-10000':  '#3245bd',
    'LIME-50':         '#c7e883',
    'LIME-100':        '#8fc543',
    'LIME-200':        '#66912d',
    'LRP':        '#008080',
    'GradCAM':         'xkcd:camel',
    'aIDG':            '#fcd459',
    'aGradExpl':       '#dd7c2b',
    'GradShap':       '#800000',
}
method_symbol = {
    'BPT-100'   :       '$\\clubsuit$1',
    'BPT-500'   :       '$\\clubsuit$2',
    'BPT-1000'  :       '$\\clubsuit$3',
    'BPTR-100'   :       '$\\bigstar$1',
    'BPTR-500'   :       '$\\bigstar$2',
    'BPTR-1000'  :       '$\\bigstar$3',
    'AA-100'    :       '$\\boxplus$1',
    'AA-500'    :       '$\\boxplus$2',
    'AA-1000'   :       '$\\boxplus$3',
    'AA-5000'   :       '$\\boxplus$4',
    'AA-10000'  :       '$\\boxplus$5',
    'LIME-50'   :       '\\textbf{0}1',
    'LIME-100'  :      '\\textbf{0}2',
    'LIME-200'  :      '\\textbf{0}3',
    'LRP'       :      '\\textbf{$\\Pi$}',
    'GradCAM'   :      '$\\bigtriangledown$',
    'aIDG'      :      '$\\divideontimes$',
    'aGradExpl' :     '$\\bigtriangleup$',
     'GradShap':       '$\\square$',
}

method_to_shortname = {}
for i,m in enumerate(method_colors.keys()):
    method_to_shortname[m] = f'{method_symbol[m]}'

In [ ]:
auc_methods      = ['aucI_pred', 'aucD_pred', 'aucI_mse', 'aucD_mse', 'max_IoU', 'au_IoU', 'time_exp']

## GENERATE/LOAD CSS AND JS FILE

In [ ]:
GENERATE_FILE = True
if GENERATE_FILE:
    with open(f'{path_utils}/html_stylesheet.txt', 'w') as f:
        f.write("""
                <style>
                    table.space {margin-top: 10px;padding-top: 10px;width: 100%;} 
                    table{border: 1px solid black;},
                    th{font-size: 2em},tr{font-size: 2em},
                    .collapsible-title {
                      font-size: 18px;
                      font-weight: bold;
                      color: #333;
                    }
                    .collapsible-text {
                      font-size: 16px;
                      color: #666;
                      line-height: 1.5;
                    }
                    .collapsible-content {
                      font-size: 20px;}
                    table {
                      text-align: left;
                      position: relative;
                      border-collapse: separate; 
                    border-top: 2px solid;
                    border-bottom: 2px solid;
                    border-right: 2px solid;
                    }
                    td {
                    padding: 0.1rem;
                    }
                    th {
                        border: 2px solid black;
                        padding: 0.1rem;
                    }
                    tr.red th {
                      background: red;
                      color: white;
                    }
                    tr.green th {
                      background: green;
                      color: white;
                    }
                    tr.purple th {
                      background: purple;
                      color: white;
                    }
                    th {
                    border-radius: 3px;
                    background-color: rgba(255, 255, 255, 0.8) !important;
                    text-align:center;
                    background: white;
                    position: sticky;
                    top: 0;
                    box-shadow: 0 2px 2px -1px rgba(0, 0, 0, 0.4);
                    }
                    tr:hover {background-color: rgba(234, 242, 215,0.8);}
                    .switch {
                    position: relative;
                    display: inline-block;
                    width: 60px;
                    height: 34px;
                    }
                    .switch input {
                    opacity: 0;
                    width: 0;
                    height: 0;
                    }
                    .slider {
                    position: absolute;
                    cursor: pointer;
                    top: 0;
                    left: 0;
                    right: 0;
                    bottom: 0;
                    background-color: #808080;
                    -webkit-transition: .4s;
                    transition: .4s;
                    }
                    .slider:before {
                    position: absolute;
                    content: "";
                    height: 26px;
                    width: 26px;
                    left: 4px;
                    bottom: 4px;
                    background-color: white;
                    -webkit-transition: .4s;
                    transition: .4s;
                    }
                    input:checked + .slider {
                    background-color: #2196F3;
                    }
                    input:focus + .slider {
                    box-shadow: 0 0 1px #2196F3;
                    }
                    input:checked + .slider:before {
                    -webkit-transform: translateX(26px);
                    -ms-transform: translateX(26px);
                    transform: translateX(26px);
                    }
                    .slider.round {
                    border-radius: 34px;
                    }
                    .slider.round:before {
                    border-radius: 50%;
                    }
                    #topBtn {
                        position: fixed;
                        bottom: 5px;
                        float: right;
                        right: 18.5%;
                        left: 0.25%;
                        max-width: 70px;
                        width: 100%;
                        font-size: 18px;
                        border-color: rgba(85, 85, 85, 0.8);
                        background-color: rgba(27, 163, 156,0.9);
                        padding: 2px;
                        border-radius: 10px;
                        }
                    #topBtn:hover {
                        background-color: #7dbbf1;
                    }
                    table {border: none;}
                    * {
                    box-sizing: border-box;
                    }
                    .box {
                    float: left;
                    width: 100%;
                    padding: 10px;
                    }
                    .clearfix::after {
                    content: "";
                    clear: both;
                    display: table;
                    }
                    table {
                    height: 100%;
                    border-collapse: seperate;
                    width: 100%;
                    margin: 10px;
                    font-size: 0.8em;
                    }
                    thead {
                    border-radius: 3px;
                    position: sticky;
                    top: 0;
                    background: #eee;
                    }
                    header h1 {
                        font-size: 70px;
                        font-weight: 600;
                        background-image: linear-gradient(to left, #553c9a, #b393d3);
                        color: transparent;
                        background-clip: text;
                        -webkit-background-clip: text;
                        text-shadow: 0px 4px 10px rgba(179, 147, 211, 0.8)
                    }
                    h1 {
                      font-size: 30px;
                      line-height: 1.8;
                      text-transform: uppercase;
                      font-family: "Montserrat", sans-serif;
                    }
                .hero {
                    position: relative;
                    background: #333 url(http://srdjanpajdic.com/slike/2.jpg) no-repeat center center fixed;
                    -webkit-background-size: cover;
                    -moz-background-size: cover;
                    background-size: cover;
                    text-align: center;
                    color: #fff;
                    padding-top: 20px;
                    min-height: 300px;
                    letter-spacing: 2px;
                    font-family: "Montserrat", sans-serif;
                    h1 {
                    font-size: 50px;
                    line-height: 1.3;
                    span {
                    font-size: 25px;
                    color: $color2;
                    border-bottom: 2px solid $color2;
                    padding-bottom: 8px;
                    line-height: 3;
                    }
                }
                .main_container{
                   width:99%;
                }
                html, body {
                height:100%;
                width:99%;
                margin:0;
                padding:0;
                }
                    </style>
                    """)
    with open(f'{path_utils}/html_head.txt', 'w') as f:
        f.write("""
            <head>
                <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css"
                    rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" 
                    crossorigin="anonymous">
                <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/js/bootstrap.bundle.min.js"
                    integrity="sha384-MrcW6ZMFYlzcLA8Nl+NtUVF0sA7MsXsP1UyJoMp4YLEuNSfAP+JcXn/tWtIaxVXM" crossorigin="anonymous">
                </script>
            </head>
            """)
    with open(f'{path_utils}/html_footer_scripts_full.txt', 'w') as f:
        f.write("""
            <script>const toggleAllButton = document.getElementById('toggleAllButton');
                const collapsibleContents = document.querySelectorAll('.collapsible-content');
                toggleAllButton.addEventListener('click', () => {
                  collapsibleContents.forEach(content => {
                    if (content.style.display === 'none') {
                      content.style.display = 'block';
                      content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                    } else {
                      content.style.display = 'none';
                      content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                    }
                  });
                });
            </script>
            """)
    with open(f'{path_utils}/html_footer_scripts_IoU.txt', 'w') as f:
        f.write("""<script>
        const toggleAllButton = document.getElementById('toggleAllButton');
            const collapsibleContents = document.querySelectorAll('.collapsible-content');
            toggleAllButton.addEventListener('click', () => {
              collapsibleContents.forEach(content => {
              document.getElementById('data_rows').style.border = 'none';
                if (content.style.display === 'none') {
                  content.style.display = 'block';
                  content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                } else {
                  content.style.display = 'none';
                  content.previousElementSibling.querySelector('.toggle-icon').textContent = '';
                }
              });
            });
            </script>
           """)

In [ ]:
with open(f"{path_utils}/html_stylesheet.txt", "r+") as f:
    html_stylesheet = f.read()
with open(f"{path_utils}/html_footer_scripts_IoU.txt", "r+") as f:
    html_footer_scripts_IoU = f.read()
with open(f"{path_utils}/html_footer_scripts_full.txt", "r+") as f:
    html_footer_scripts_full = f.read()
with open(f"{path_utils}/html_head.txt", "r+") as f:
    html_head = f.read()

## HTML FULL

In [ ]:



sel_thresh  = 1.01
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
nf_images = []
print(f'Total Images: {total_images}\t Total Methods {len(methods)}')
######################################################################################################
if use_subset:
    files_set = files[:subset_count]
else:
    files_set = files
html_f_name = f'{path_html}/HTML_{exp_no}_{model_name}_{background_type}_{len(files_set)}.html'
######################################################################################################
if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all IoU results [Visual(heatmaps) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_name}, background_type: {background_type}] - No of Images:{total_images}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')
        
for fff_id,file_n in enumerate(tqdm(files_set)):
    # file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    img_path = f'plots//{file_n}_{background_type}_heatmap.png'
    # img_path = f'{plots_path}//{file_n}_{background_type}_heatmap.png'
    
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                    <p>{file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('''
                    </div>
                </div>
                </td></tr>
                ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

In [ ]:
print(f'Total images:\t\t{len(files):<25}\nMatched images:\t\t{total_images}\nUnmatched images\t{len(nf_images)}')
print(f'Data Written in File :\t{html_f_name}')

## IoU HTML FILE

In [ ]:


sel_thresh  = 1.01
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
auc_methods_sel = auc_methods[-3:-1]
################################################################################################
if use_subset:
    files_set = files[:subset_count]
else:
    files_set = files
html_f_name = f'{path_html}/HTML_IoU_{exp_no}_{model_name}_{background_type}_{len(files_set)}.html'
################################################################################################
print(f'Total Images: {total_images}\t Total Methods {len(methods)}')

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('\
    <html>\
    ')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                <h1><span>Webpage containing all IoU results [Visual(IoU) and Numerical] for </span>
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_type}, background_type: {background_type}] - No of Images:{total_images}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
# (0.0, 0.0, 0.75) # True Positives
# (1.0, 0.6, 0.2) # False Positives
# (1.0, 0.4, 1.0) # False Negatives
with open(html_f_name, 'a') as f:
        f.write('''
        <fieldset>
            <legend>Toggle Explanation Details:</legend>
            <label class="switch">
                  <input type="checkbox" id="toggleAllButton" checked>
                  <span class="slider round"></span>
            </label>
        </fieldset>
        <br>
        ''')
        f.write(f'\
        <table id="table-1"><tr>\
            <thead>\
            <tr>\
            <div class="clearfix">\
                <th colspan="{len(methods)+2}">\
                    <div class="box">\
                        LEGEND\
                    </div>\
                </th>\
            </tr>\
            <tr>\
                <th colspan=4>\
                    <div class="box" style="background-color:#0000bf; color: #ffffff;">\
                        True Positive\
                    </div>\
                </th>\
                <th colspan=5>\
                    <div class="box" style="background-color:#ff9933">\
                        False Negative\
                    </div>\
                </th>\
                <th colspan=5>\
                    <div class="box" style="background-color:#ff66ff">\
                        False Positive\
                    </div>\
                </th>\
                </th>\
            </div>\
        </tr>\
        <tr>\
        <th colspan="{len(methods)+2}">\
            <div class="box">\
                Methods\
            </div>\
        </th>\
        </tr>\
            <tr>\
            <th align= "center"> Input Image</th> \
            <th align= "center"> GrTruth</th> \
                ')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>\
        ')

with open(html_f_name, 'a') as f:
    f.write('\
    </thead>\
    ')

for fff_id,file_n in enumerate(tqdm(files_set)):
    # file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        # print(f'image not found - {file_n}')
        continue

    # img_path = f'plots_IoU//{file_n}_{background_type}.png'
    img_path = f'plots_IoU/{file_n}_{background_type}_IoU.png'
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                    <p>{file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods_sel:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td id="data_rows"> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('\
                    </div>\
                </div>\
                </td></tr>\
                ')
with open(html_f_name, 'a') as f:
    f.write('''
                </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_IoU)
    f.write('''
                    </div>
                </body>
            </html>
            ''')
print(f'Data Written in File :\tfile://{os.path.abspath(html_f_name)}')

In [ ]:
print(f'Total images:\t\t{len(files):<25}\nMatched images:\t\t{total_images}\nUnmatched images\t{len(nf_images)}')
print(f'Data Written in File :\t{html_f_name}')

# HEATMAPS & IOU Together

In [ ]:
# import os

# directory_path = os.path.join(path_html, 'plots')

# files = [f.split('.')[0].split('_')[0] for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# print(len(files),files)

In [ ]:
sel_thresh  = 1.01
version = 'full' if sel_thresh>1 else 'low'
total_images = len(np.unique(df[df.f_S<sel_thresh].image))
nf_images = []
print(f'Total Images: {total_images}\t Total Methods {len(methods)}')

################################################################################################
if use_subset:
    files_set = files[:subset_count]
    # directory_path = os.path.join(path_html, 'plots')
    # files_set = [f.split('.')[0].split('_')[0] for f in os.listdir(plots_path) if os.path.isfile(os.path.join(plots_path, f))]

else:
    files_set = files
html_f_name = f'{path_html}/HTML_{exp_no}_{model_name}_{background_type}_Combined_{len(files_set)}.html'
################################################################################################

if os.path.exists(html_f_name):
    os.remove(html_f_name)
with open(html_f_name, 'a') as f:
    f.write('<html>')
    f.write(html_head)
    f.write('''
        <body>
        <div class='main_container'>
            <div class="hero">
                    <br>Shapley image explanations<br>
                    with data-aware Binary Partition Trees.
                </h1>
                ''')
    f.write(f'<h3>Params: [model: {model_name}, background_type: {background_type}] - No of Images:{len(files_set)}</h3>')
    f.write('''
                <div class="mouse">
                    <span></span>
                </div>
            </div>
            ''')
    f.write(html_stylesheet)
with open(html_f_name, 'a') as f:
        f.write('''
                <fieldset>
                    <legend>Toggle Explanation Details:</legend>
                    <label class="switch">
                          <input type="checkbox" id="toggleAllButton" checked>
                          <span class="slider round"></span>
                    </label>
                </fieldset>
                <br><br>
                <table id="table-1"><tr>
                    <th align= "center"> Input Image</th> 
                    <th align= "center"> GrTruth</th> 
                ''')
for meth in methods:
    if meth in ['AA-100','AA-500']:
        # print(meth)
        meth = f'&nbsp{meth}&nbsp'
    meth= '|GrExpl|' if meth=='GradExpla' else meth
    meth= '  GrCAM  ' if meth=='GradCAM' else meth
    with open(html_f_name, 'a') as f:
        f.write(f'<th align= "center"> {meth}</th>')
        
for fff_id,file_n in enumerate(tqdm(files_set)):
    # file_n = fff.split('.')[0].split('//')[-1]
    if file_n not in df.image.values:
        nf_images.append(file_n)
        continue
    # img_path = f'plots//{file_n}_{background_type}_heatmap.png'
    img_path = f'{plots_path}//{file_n}_{background_type}_heatmap.png'
    img_iou_path = f'{plotsIoU_path}//{file_n}_{background_type}_IoU.png'
    
    PP = df[df.image==file_n].f_S.iloc[0]
    # if version=='low':
    #     if PP>0.5:
    #         continue
    PL = df[df.image==file_n].pred_lbl.iloc[0]
    with open(html_f_name, 'a') as f:
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_path} width="99.9%"> <br>\
                    <p style="font-size:22">{file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
        f.write(f'<tr><td colspan="{len(methods)+2}">\
                <img src={img_iou_path} width="99.9%"> <br>\
                    <p style="font-size:22">{file_n}  -  {PL}  -  {PP:0.5}  -  \
                    </p>\
                </td>\
                ')
    with open(html_f_name, 'a') as f:
            f.write(f'\
                <tr><td">\
                  ')
    for sm in auc_methods:
        df1 = df[(df.image==file_n)].eval(f'{sm}')
        with open(html_f_name, 'a') as f:
            f.write(f'\
                    <tr><td>\
                    <div class="collapsible-container">\
                      <div class="collapsible-header">\
                        <span class="toggle-icon"></span>\
                      </div>\
                      <div class="collapsible-content">\
                        <p>\
                        <b>{sm}</b>\
                        </div>\
                    </div>\
                    </td>\
                    ')
        for meth_id,meth in enumerate(methods):
            df2 = df1.iloc[meth_id]
            with open(html_f_name, 'a') as f:
                f.write(f'\
                        <td> \
                        <div class="collapsible-container">\
                          <div class="collapsible-header">\
                            <span class="toggle-icon"></span>\
                          </div>\
                          <div class="collapsible-content">\
                            <p>\
                            {df2:0.5} </p>\
                            </div>\
                        </div>\
                        </td>\
                        ')
        with open(html_f_name, 'a') as f:
            f.write(f'\
            </tr>\
            ')
    with open(html_f_name, 'a') as f:
        f.write('''
                    </div>
                </div>
                </td></tr>
                ''')
with open(html_f_name, 'a') as f:
    f.write('''
            </tr>
            </table>
            <button id="topBtn"><a href="#top" style="color: white">Top</a></button>
            ''')
    f.write('<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>')
    f.write(html_footer_scripts_full)
    f.write('''  
                </body>
            </html>
            ''')
print(f'Data Written in File :\t{html_f_name}')

# END